This Notebook is used for testing code. Content can be ignored for now

In [ ]:
import platform
import os

if platform.system() == "Darwin":  # macOS
    path = os.environ.get("DYLD_FALLBACK_LIBRARY_PATH", "")
    path += (":" if path else "") + "/opt/homebrew/lib"
    os.environ["DYLD_FALLBACK_LIBRARY_PATH"] = path
elif platform.system() == "Windows":  # Windows
    OPENSLIDE_PATH = r"C:\\Users\\mlser\\openslide-bin-4.0.0.6-windows-x64\\openslide-bin-4.0.0.6-windows-x64\\bin"
    if hasattr(os, "add_dll_directory"):
        _openslide_dll_handle = os.add_dll_directory(str(OPENSLIDE_PATH))
    else:
        os.environ["PATH"] += os.pathsep + str(OPENSLIDE_PATH)

# Third Party
try:
    # Third Party
    import openslide
except ImportError as e:
    raise ImportError(
        "OpenSlide could not be imported. Ensure the library is installed and the path is set correctly."
    ) from e

# CuBATS
import cubats.slide_collection.registration as registration
from cubats.slide_collection.slide_collection import SlideCollection
from cubats.slide_collection import segmentation
import cubats.slide_collection.segmentation as segmentation
from cubats import cutils as cutils
from cubats.slide_collection.slide import Slide
from cubats.slide_collection import tile_quantification
from cubats.slide_collection import tile_colocalization

In [ ]:
import os
import glob

main_dir = r"F:\CuBATS_out"

# files to keep
keep_files = {
    "quantification.pickle",
    "dual_antigen_expressions.pickle",
    "triplet_antigen_expressions.pickle",
}

for name in os.listdir(main_dir):
    pickle_dir = os.path.join(main_dir, name, "data", "pickle")
    if os.path.isdir(pickle_dir):
        pickle_files = glob.glob(os.path.join(pickle_dir, "*.pickle"))
        for f in pickle_files:
            if os.path.basename(f) in keep_files:
                print(f"Keeping: {f}")
                continue
            try: 
                os.remove(f)
                print(f"Deleted: {f}")
            except Exception as e:
                print(f"Could not delete {f}: {e}")

In [ ]:
def plot_tile_quantification_results(self, tilename, img_true=True, numeric=True):
    """Plots quantification results for a tile.

    This function plots quantification results of a given tilename. It plots the DAB-image, the histogram of the
    intensity distribution, a bar plot containing the amount of pixels attributed to each zone and numeric results.
    Display of the image and numeric results are optional, default is set to True. Images can only be display if they
    have been saved during quantification in functions: quantify_all_slides or quantify_single_slide.

    Args:
        tilename (str): name of tile (col_row)
        img_true (bool, optional): Plot DAB image. Defaults to True.
        numeric (bool, optional): include numeric information. Defaults to True.
    """
    names = []
    images = []
    hists = []
    hist_centers = []
    zones = []
    percentages = []
    scores = []
    px_count = []
    img_idx = 0
    hist_idx = 1
    bar_idx = 2
    numeric_idx = 3

    if img_true and numeric:
        for dir in self.dab_tile_dir_list:
            file = os.path.join(dir, (tilename + "_DAB.tif"))
            if os.path.exists(file):
                try:
                    img = skimage.io.imread(file)
                    images.append(img)
                except Exception as e:
                    print(f"Error reading image {file}: {e}")
        fig, ax = plt.subplots(
            4,
            self.quantification_results_list.__len__(),
            figsize=(self.quantification_results_list.__len__() * 5, 18),
        )
    elif img_true and not numeric:
        for dir in self.dab_tile_dir_list:
            file = os.path.join(dir, (tilename + "_DAB.tif"))
            if os.path.exists(file):
                img = skimage.io.imread(file)
                images.append(img)
        fig, ax = plt.subplots(
            3,
            self.quantification_results_list.__len__(),
            figsize=(self.quantification_results_list.__len__() * 5, 13),
        )
    elif not img_true and numeric:
        fig, ax = plt.subplots(
            3,
            self.quantification_results_list.__len__(),
            figsize=(self.quantification_results_list.__len__() * 5, 13),
        )
        hist_idx = 0
        bar_idx = 1
        numeric_idx = 2

    else:
        fig, ax = plt.subplots(
            2,
            self.quantification_results_list.__len__(),
            figsize=(self.quantification_results_list.__len__() * 5, 9),
        )
        hist_idx = 0
        bar_idx = 1

    tile_exists = False

    for i in range(self.quantification_results_list.__len__()):
        for j in range(self.quantification_results_list[i][1].__len__()):
            if self.quantification_results_list[i][1][j]["Tilename"] == tilename:
                names.append(self.quantification_results_list[i][0])
                hists.append(
                    self.quantification_results_list[i][1][j]["Histogram"])
                hist_centers.append(
                    self.quantification_results_list[i][1][j]["Hist_centers"]
                )
                zones.append(
                    self.quantification_results_list[i][1][j]["Zones"])
                percentages.append(
                    self.quantification_results_list[i][1][j]["Percentage"]
                )
                scores.append(
                    self.get_score_name(
                        self.quantification_results_list[i][1][j]["Score"].tolist(
                        )
                    )
                )
                px_count.append(
                    self.quantification_results_list[i][1][j]["Px_count"])
                tile_exists = True
                break

    if not tile_exists:
        raise ValueError(
            "The given tilename does not exist for one or more of the slides. \
                Please make sure to select an existing tilename."
        )

    max_y_hist = round(max([max(hist) for hist in hists]), -4) + 10000
    max_y_zone = round(max([max(zone) for zone in zones[:4]]), -4) + 20000

    for i in range(self.quantification_results_list.__len__()):
        if img_true:
            ax[img_idx, i].imshow(images[i])
            ax[img_idx, i].set_title("Slide: " + names[i])
            ax[img_idx, i].axis("off")
        # Histogram
        ax[hist_idx, i].plot(hist_centers[i], hists[i], lw=2)
        ax[hist_idx, i].set_title("Quantification: " + names[i])
        ax[hist_idx, i].set_xlabel("Pixel Intensity")
        ax[hist_idx, i].set_ylabel("Number of Pixels")
        ax[hist_idx, i].set_xlim([0, 255])
        ax[hist_idx, i].set_ylim([0, max_y_hist])
        ax[hist_idx, i].axvline(x=60, color="r", linestyle="--")
        ax[hist_idx, i].axvline(x=120, color="r", linestyle="--")
        ax[hist_idx, i].axvline(x=180, color="r", linestyle="--")
        ax[hist_idx, i].axvline(x=235, color="r", linestyle="--")
        # Bar Plot
        ax[bar_idx, i].bar(
            ["High Positive", "Positive", "Low Positive", "Negative"], zones[i][:4]
        )
        ax[bar_idx, i].set_title("Scoring: " + names[i])
        ax[bar_idx, i].set_ylim([0, max_y_zone])
        ax[bar_idx, i].set_ylabel("Number of Pixels")
        # Numeric Results
        if numeric:
            rows = [
                "High Positive",
                "Positive",
                "Low Positive",
                "Negative",
                "Total Pixel Count",
                "Score",
            ]
            cell_text = [
                [str(round(percentages[i][0], 2)) + "%"],
                [str(round(percentages[i][1], 2)) + "%"],
                [str(round(percentages[i][2], 2)) + "%"],
                [str(round(percentages[i][3], 2)) + "%"],
                [px_count[i]],
                [scores[i]],
            ]
            ax[numeric_idx, i].axis("off")
            ax[numeric_idx, i].table(
                cellText=cell_text,
                colWidths=[0.5] * 3,
                rowLabels=rows,
                loc="best",
            )
            ax[numeric_idx, i].set_title("Numeric Results: " + names[i])

    fig.suptitle(f"Quantification Results for Tile: {tilename}\n", fontsize=16)
    fig.tight_layout()
    plt.show()


In [ ]:
import os
import glob
main_dir = r"F:\CuBATS_out"

for name in os.listdir(main_dir):
    pickle_dir = os.path.join(main_dir, name, "data", "pickle")
    if os.path.isdir(pickle_dir):
        pickle_files = glob.glob(os.path.join(pickle_dir, "*.pickle"))
        for f in pickle_files:
            try: 
                os.remove(f)
                print(f"Deleted: {f}")
            except Exception as e:
                print(f"Could not delete {f}: {e}")

In [ ]:
import os
import shutil
main_dir = r"F:\CuBATS_out"
files_to_move = [
            "pixel-level_quantification_results.csv",
            "pixel-level_dual_antigen_expressions.csv",
            "pixel-level_triplet_antigen_expressions.csv",
        ]
for name in os.listdir(main_dir):
    data_dir = os.path.join(main_dir,name,"data")
    if not os.path.isdir(data_dir):
        continue
    
    for fname in files_to_move:
        file_path = os.path.join(data_dir, fname)
        if not os.path.isfile(file_path):
            print(name)

In [ ]:
import os
import shutil
main_dir = r"F:\CuBATS_out"
files_to_move = [
    "quantification_results.csv",
    "dual_antigen_expressions.csv",
    "triplet_antigen_expressions.csv",
]
for name in os.listdir(main_dir):
    data_dir = os.path.join(main_dir,name,"data")
    if not os.path.isdir(data_dir):
        continue

    tile_level_dir = os.path.join(data_dir,"pixel_level_old")
    os.makedirs(tile_level_dir,exist_ok=True)

    for fname in files_to_move:
        file_path = os.path.join(data_dir, fname)
        if os.path.isfile(file_path):
            dest = os.path.join(tile_level_dir, fname)
            shutil.move(file_path, dest)
            print(f"File exists:  {file_path} ")
        else:
            print(f"File not found: {file_path}")


In [ ]:
import os
import shutil

base_folder = r"F:\CuBATS_out"  # <- Change this

folders_to_remove = ["colocalization", "reconstructed_slides", "tiles"]
files_to_move = [
    "quantification_results.csv",
    "dual_antigen_expressions.csv",
    "triplet_antigen_expressions.csv",
]

for folder_name in os.listdir(base_folder):
    folder_path = os.path.join(base_folder, folder_name)
    if not os.path.isdir(folder_path):
        continue

    # Remove empty folders
    for subfolder in folders_to_remove:
        subfolder_path = os.path.join(folder_path, subfolder)
        if os.path.isdir(subfolder_path) and not os.listdir(subfolder_path):
            os.rmdir(subfolder_path)
            print(f"Removed empty folder: {subfolder_path}")

    # Move files inside data/old
    data_dir = os.path.join(folder_path, "data")
    if os.path.isdir(data_dir):
        old_dir = os.path.join(data_dir, "old")
        os.makedirs(old_dir, exist_ok=True)
        for fname in files_to_move:
            file_path = os.path.join(data_dir, fname)
            if os.path.isfile(file_path):
                dest = os.path.join(old_dir, fname)
                shutil.move(file_path, dest)
                print(f"Moved file {file_path} to {dest}")

In [ ]:
import logging

# Set up logging
logging.basicConfig(filename='registration.log', level=logging.INFO, format='%(asctime)s %(message)s')

# Define the source and destination directories
patient_data_dir = r"E:\patient_data"
valis_out_dir = r"E:\valis_out"

# Define the required space in bytes (e.g., 100 GB)
required_space_per_folder = 80 * 1024 * 1024 * 1024  # 100 GB
total_available_space = 590 * 1024 * 1024 * 1024  # 530 GB

# Calculate the maximum number of folders that can be processed
max_folders = total_available_space // required_space_per_folder
print(f"Maximum number of folders that can be processed: {max_folders}")

def has_enough_space(path, required_space):
    """Check if there is enough space available at the given path."""
    total, used, free = shutil.disk_usage(path)
    logging.info(f"Checking space: Total={total}, Used={used}, Free={free}")
    return free >= required_space

# Get the list of folders in patient_data
patient_folders = [f for f in os.listdir(patient_data_dir) if os.path.isdir(os.path.join(patient_data_dir, f))]

# Initialize the counter for processed folders
processed_folders = 0

# Iterate over each folder
for folder_name in patient_folders:
    if processed_folders >= max_folders:
        break

    slide_src_dir = os.path.join(patient_data_dir, folder_name)
    results_dst_dir = os.path.join(valis_out_dir, folder_name, "valis")
    registered_slide_dest_dir = os.path.join(results_dst_dir, "registered_slides")

    # Check if the folder name is not in valis_out
    if not os.path.exists(results_dst_dir):
        # Create the destination directory and the "valis" folder inside it
        os.makedirs(results_dst_dir, exist_ok=True)

        # Check if there is enough space available
        if has_enough_space(valis_out_dir, required_space_per_folder):
            try:
                # Perform the registration
                logging.info(f"Starting registration for {folder_name}")
                registration.register_slides(slide_src_dir, slide_src_dir, microregistration=True, max_non_rigid_registration_dim_px=2000
                                    )
                logging.info(f"Completed registration for {folder_name}")
                processed_folders += 1
            except Exception as e:
                logging.error(f"Error processing {folder_name}: {e}")
                print(f"Error processing {folder_name}. Check the log for details.")
        else:
            logging.warning(f"Not enough space available for {folder_name}. Skipping registration.")
            print(f"Not enough space available for {folder_name}. Skipping registration.")
    else:
        logging.info(f"Results already exist for {folder_name}. Skipping registration.")
        print(f"Results already exist for {folder_name}. Skipping registration.")

print(f"Processed {processed_folders} folders.")

In [ ]:
slide_src_dir = r"E:\processed\N_2016_000520"
results_dst_dir = r"E:\valis_out"
registered_slide_dest_dir = r"E:\valis_out/N_2016_000520/registered_slides"
import valis
from cubats.slide_collection import registration

In [ ]:
registration.register_slides(slide_src_dir,results_dst_dir,microregistration=True,max_non_rigid_registration_dim_px=2000)

In [ ]:
registrar = valis.registration.Valis(slide_src_dir, results_dst_dir)
rigid_arr, non_rigid, errodf = registrar.register()

In [ ]:
registrar.register_micro(max_non_rigid_registration_dim_px=2000)

In [ ]:
import pyvips
registrar = valis.registration.load_registrar(r"E:\valis_out\N_2016_000317\valis\data\_registrar.pickle")
pyvips.leak_set(True)
registrar.warp_and_save_slides(registered_slide_dest_dir, crop ="overlap")
# for name, slide_obj in registrar.slide_dict.items():
#     dsf = os.path.join(registered_slide_dest_dir,f"{name}.ome.tiff")
#     slide_obj.warp_and_save_slide(dst_f = dsf, crop = "overlap", compression= "WEBP")

In [ ]:
valis.registration.kill_jvm()

In [ ]:
registration.register_slides(slide_src_dir, results_dst_dir, microregistration= True, max_non_rigid_registration_dim_px=2000)

In [ ]:
registration.register_slides_high_resolution(slide_src_dir, slide_src_dir, 1.0)

In [ ]:
registration.register_slides(
    slide_src_dir,
    results_dst_dir,
    microregistration=True,
    max_non_rigid_registration_dim_px=2000
)

In [ ]:
path = r"E:\valis_out\N_2016_000167\registered_slides\N_2016_000167_HE.ome.tiff"
n_path = r"C:\Users\mlser\Desktop\ml_model\seg_mod_256_2023-02-15.onnx"
#path = '/Volumes/EXTERNAL_USB/valis out/N_2014_000862/registered_slides/N_2014_000862_HE.ome.tiff'
#n_path = '/Users/moritz.lokal/Downloads/new_model_w_normalizer/seg_monai_1024_2024-10-17.onnx'
tile_size = (512, 512)
segmentation.run_tumor_segmentation(path, n_path, tile_size, output_path=None, normalization=False, inversion=False, plot_results=True)

In [ ]:
src_dir= r"H:\valis_out\N_2018_000671\registered_slides"
dst_dir = r"F:\CuBATS_out\N_2018_000671"
path_antigen_profiles = r"C:\Users\mlser\Desktop\CuBATS\src\cubats\assets\antigen_profiles.json"
#segmentation.run_tumor_segmentation(he_slide, model, (512, 512), output_path=None, normalization=False, inversion=False, plot_results=False)
slide_collection = SlideCollection("N_2018_000671", src_dir, dst_dir, path_antigen_profiles=path_antigen_profiles)


In [ ]:
slide_collection.quantification_results

In [ ]:
slide_collection.quantify_all_slides(masking_mode='pixel-level')
slide_collection.quantification_results

In [ ]:
slide_collection.get_dual_antigen_combinations(masking_mode = 'pixel-level')
slide_collection.dual_antigen_results

In [ ]:
slide_collection.get_triplet_antigen_combinations(masking_mode='pixel-level')
slide_collection.triplet_antigen_results

In [ ]:
b7h3 = slide_collection.slides[1].name
slide_collection.quantify_single_slide(b7h3,save_img=False, masking_mode='pixel-level')
slide_collection.quantification_results

In [ ]:
gd2 = slide_collection.slides[2].name
slide_collection.quantify_single_slide(gd2,save_img=False, masking_mode='pixel-level')
slide_collection.quantification_results

In [ ]:
slide_collection.slides[5].tiles.get_tile()

In [ ]:
pdpn = slide_collection.slides[5].name
slide_collection.quantify_single_slide(pdpn,save_img=False, masking_mode='pixel-level')
slide_collection.quantification_results

In [ ]:
nz1 = slide_collection.slides[6].name
slide_collection.quantify_single_slide(nz1,save_img=False, masking_mode='pixel-level')
slide_collection.quantification_results

In [ ]:
ror2 = slide_collection.slides[7].name
slide_collection.quantify_single_slide(ror2,save_img=False, masking_mode='pixel-level')
slide_collection.quantification_results

In [ ]:
slide_collection.get_dual_antigen_combinations()

In [ ]:
slide_collection.dual_antigen_results

In [ ]:
slide_collection.compute_dual_antigen_combination(slide_collection.slides[0], slide_collection.slides[1], save_img=False)

In [ ]:
slide_collection.dual_antigen_results

In [ ]:
slide_collection.slides[0].detailed_quantification_results

In [ ]:
slide_collection.quantify_all_slides(masking_mode='pixel-level')

In [ ]:
slide_collection.quantification_results

In [ ]:
slide_collection.get_dual_antigen_combinations()
#slide_collection.get_triplet_antigen_combinations()

In [ ]:
slide_collection.dual_antigen_results

In [ ]:
import os
import pathlib

path = r"F:\CuBATS_out\N_2017_000127\data"

print(f"[DEBUG] Raw path: {repr(path)}")
print(f"[DEBUG] os.path.exists: {os.path.exists(path)}")
print(f"[DEBUG] os.path.isdir: {os.path.isdir(path)}")
print(f"[DEBUG] os.path.isfile: {os.path.isfile(path)}")
print(f"[DEBUG] pathlib.Path exists: {pathlib.Path(path).exists()}")
print(f"[DEBUG] Directory listing of parent:")
parent = pathlib.Path(path).parent
try:
    for item in parent.iterdir():
        print(" -", item)
except Exception as e:
    print("Could not list directory:", e)

In [ ]:
slide_collection.quantification_results

In [ ]:
#slide_collection.quantify_all_slides()
slide_collection.get_dual_antigen_combinations()
slide_collection.get_triplet_antigen_combinations()


In [ ]:
main_folder = r"F:\CuBATS_out"

for folder_name in os.listdir(main_folder):
    folder_path = os.path.join(main_folder, folder_name)
    if not os.path.isdir(folder_path):
        continue  # Skip if not a folder

    data_path = os.path.join(folder_path, "data")
    quant_file = os.path.join(data_path, "quantification_results.csv")
    dual_path = os.path.join(data_path, "dual_antigen_expressions.csv")
    triplet_path = os.path.join(data_path, "triplet_antigen_expressions.csv")

    # Check for all required contents
    if os.path.isfile(quant_file) and (os.path.isfile(dual_path) and os.path.isfile(triplet_path)):
        continue  # Skip valid folders

    # If any requirement is missing, print the folder name
    print(f"Missing required content in: {folder_name}")

In [ ]:
src_dir= r"F:\valis\N_2019_000955\registered_slides"
dst_dir = r"F:\CuBATS_out\N_2019_000955"
path_antigen_profiles = r"C:\Users\mlser\Desktop\CuBATS\src\cubats\assets\antigen_profiles.json"
#segmentation.run_tumor_segmentation(he_slide, model, (512, 512), output_path=None, normalization=False, inversion=False, plot_results=False)
slide_collection = SlideCollection("N_2019_000955", src_dir, dst_dir, path_antigen_profiles=path_antigen_profiles)


In [ ]:
slide_collection.slides[6].name

In [ ]:
slide_collection.compute_dual_antigen_combination(slide1=slide_collection.slides[2], slide2=slide_collection.slides[6],save_img=True)

In [ ]:
slide_collection.slides[0].reconstruct_slide(r"F:\CuBATS_out\N_2020_000706\colocalization\N_2020_000706_B7H3_and_N_2020_000706_ADAM9",r"F:\CuBATS_out\N_2020_000706\reconstructed_slides/")


In [ ]:

openslide.OpenSlide(r"F:\valis\N_2019_000955\registered_slides\N_2019_000955_PDPN.ome.tiff").get_thumbnail((512,512)).save(r"I:\images/N_2019_000955_PDPN_thumb.png","PNG")

In [ ]:
openslide.OpenSlide(r"E:\processed\N_2019_000955\N_2019_000955_PDPN.mrxs").get_thumbnail((512,512)).save(r"I:\images/N_2019_000955_B7H3_thumb_orig.png","PNG")

In [ ]:
openslide.OpenSlide(r"E:\processed\N_2019_000955\N_2019_000955_HE.mrxs").get_thumbnail((512,512)).save(r"I:\images/N_2019_000955_he_thumb_orig.png","PNG")

In [ ]:
src_dir = r"F:\valis\N_2019_000955\registered_slides"
dst_dir = r"F:\CuBATS_out\N_2019_000955"
path_antigen_profiles = r"C:\Users\mlser\Desktop\CuBATS\src\cubats\assets\antigen_profiles.json"

slide_collection = SlideCollection("N_2019_000955", src_dir, dst_dir, path_antigen_profiles=path_antigen_profiles)

In [ ]:
from PIL import Image
tile = Image.open(r"I:\images\27_20_original.tif")
res = tile_processing.quantify_tile([27,20,tile,r"I:\images",True,{
                "Name": "default",
                "low_positive_threshold": 181,
                "medium_positive_threshold": 121,
                "high_positive_threshold": 61,
            }])

In [ ]:
import numpy as np
tile = Image.open(r"I:\images\16_14_DAB.tif").convert("L")
tile = np.array(tile)
high_mask = tile < 61
med_mask = (tile>=61)&(tile<121)
low_mask = (tile>=121)& (tile<181)
neg_mask = (tile>=181)
colored_img = np.full((1024,1024), 255, dtype=np.uint8)
colored_img[high_mask]= 50
colored_img[med_mask]= 110
colored_img[low_mask]=150
Image.fromarray(colored_img).save(r"I:\images\16_14_DAB_gray_quant.tif")
high = np.full((1024,1024), 255, dtype=np.uint8)
high[high_mask] = 64
Image.fromarray(high).save(r"I:\images\16_14_DAB_high.tif")
med = np.full((1024,1024), 255, dtype=np.uint8)
med[med_mask] = 128
Image.fromarray(med).save(r"I:\images\16_14_DAB_med.tif")
low = np.full((1024,1024), 255, dtype=np.uint8)
low[low_mask]=192
Image.fromarray(low).save(r"I:\images\16_14_DAB_low.tif")


In [ ]:
slide_collection.quantify_single_slide("N_2019_000955_ADAM9",save_img=True, masking_mode=True)

In [ ]:
slide_collection.slides[3].reconstruct_slide(r"F:\CuBATS_out\N_2019_000955\tiles\N_2019_000955_ADAM9\dab",r"F:\CuBATS_out\N_2019_000955\reconstructed_slides")
openslide.OpenSlide(r"F:\CuBATS_out\N_2019_000955\reconstructed_slides\N_2019_000955_ADAM9_reconst.tif").get_thumbnail((512,512)).save(r"I:\images/N_2019_000955_adam9_thumb_masked.png","PNG")

In [ ]:
openslide.OpenSlide(r"F:\CuBATS_out\N_2019_000955\reconstructed_slides\N_2019_000955_ADAM9_reconst.tif").get_thumbnail((512,512)).save(r"I:\images/N_2019_000955_adam9_thumb_masked.png","PNG")

In [ ]:
#slide_collection.slides[3].reconstruct_slide(r"F:\CuBATS_out\N_2019_000955\tiles\N_2019_000955_PDPN\dab",r"F:\CuBATS_out\N_2019_000955\reconstructed_slides")
openslide.OpenSlide(r"F:\CuBATS_out\N_2019_000955\reconstructed_slides\N_2019_000955_PDPND240_reconst.tif").get_thumbnail((512,512)).save(r"I:\images/N_2019_000955_pdpn_thumb_masked.png","PNG")

In [ ]:
for slide in slide_collection.slides:
    if slide.antigen_profile is not None: 
        print(f"Slidename: {slide.name}, profile: {slide.antigen_profile}")

In [ ]:
slide_collection.add_antigen_profiles(r"C:\Users\mlser\Desktop\CuBATS\src\cubats\assets\antigen_profiles.json")

In [ ]:
for slide in slide_collection.slides:
    if slide.antigen_profile is not None: 
        print(f"Slidename: {slide.name}, profile: {slide.antigen_profile}")

In [ ]:
slide_collection.quantify_all_slides()

In [ ]:
slide_collection.slides[6].name

In [ ]:
slide_collection.quantify_single_slide(slide_collection.slides[6].name)

In [ ]:
slide_collection.get_dual_antigen_combinations()
slide_collection.get_triplet_antigen_combinations()

In [ ]:
slide_collection.quantification_results

In [ ]:
slide_collection.dual_antigen_results

In [ ]:
test  = slide_collection.mask.tiles.get_tile(17,(50,60)) 

In [ ]:
import numpy as np

m = np.array(test)
pos = np.sum(m == 0)
neg = np.sum(m == 255)
print(f"pos: {pos}")
print(f"neg: {neg}")
print(f"Pos/Neg: {pos/neg}")


In [ ]:
he_slide = r"E:\valis_out\N_2016_000482\valis\N_2016_000482\registered_slides\N_2016_000482_HE.ome.tiff"
model = r"C:\Users\mlser\Desktop\ml_model\seg_mod_256_2023-02-15.onnx"
src_dir= r"E:\valis_out\N_2016_000482\valis\N_2016_000482\registered_slides"
dst_dir = r"E:\valis_out\N_2016_000482"
segmentation.run_tumor_segmentation(he_slide, model, (512, 512), output_path=None, normalization=False, inversion=False, plot_results=False)
slide_collection = SlideCollection("N_2016_000482", src_dir, dst_dir)
slide_collection.quantify_all_slides()
#slide_collection.quantification_results
slide_collection.get_dual_antigen_combinations()
slide_collection.get_triplet_antigen_combinations()

In [ ]:
he_slide = r"E:\valis_out\N_2016_000686\valis\N_2016_000686\registered_slides\N_2016_000686_HE.ome.tiff"
model = r"C:\Users\mlser\Desktop\ml_model\seg_mod_256_2023-02-15.onnx"
src_dir= r"E:\valis_out\N_2016_000686\valis\N_2016_000686\registered_slides"
dst_dir = r"E:\valis_out\N_2016_000686"
segmentation.run_tumor_segmentation(he_slide, model, (512, 512), output_path=None, normalization=False, inversion=False, plot_results=False)
slide_collection = SlideCollection("N_2016_000686", src_dir, dst_dir)
slide_collection.quantify_all_slides()
#slide_collection.quantification_results
slide_collection.get_dual_antigen_combinations()
slide_collection.get_triplet_antigen_combinations()

In [ ]:
he_slide = r"E:\valis_out\N_2016_000701\valis\N_2016_000701\registered_slides\N_2016_000701_HE.ome.tiff"
model = r"C:\Users\mlser\Desktop\ml_model\seg_mod_256_2023-02-15.onnx"
src_dir= r"E:\valis_out\N_2016_000701\valis\N_2016_000701\registered_slides"
dst_dir = r"E:\valis_out\N_2016_000701"
#segmentation.run_tumor_segmentation(he_slide, model, (512, 512), output_path=None, normalization=False, inversion=False, plot_results=False)
slide_collection = SlideCollection("N_2016_000701", src_dir, dst_dir)
slide_collection.quantify_all_slides()
#slide_collection.quantification_results
slide_collection.get_dual_antigen_combinations()
slide_collection.get_triplet_antigen_combinations()

In [ ]:
src_dir= r"G:\organoid_out\N23_861\registered_slides"
dst_dir = r"G:\organoid_out\N23_861\cubats"
slide_collection = SlideCollection("N23_861", src_dir, dst_dir)
slide_collection.quantify_all_slides()
slide_collection.quantification_results

In [ ]:
slide_collection.get_dual_antigen_combinations()
slide_collection.get_triplet_antigen_combinations()

In [ ]:
src_dir= r"E:\valis_out\N_2015_001043\valis\registered_slides"
dst_dir = r"E:\valis_out\N_2015_001043"
slide_collection = SlideCollection("N_2015_001043", src_dir, dst_dir)
slide_collection.quantify_all_slides()
slide_collection.quantification_results

In [ ]:
src_dir= r"E:\valis_out\N_2016_000008\valis\registered_slides"
dst_dir = r"E:\valis_out\N_2016_000008"
slide_collection = SlideCollection("N_2016_000008", src_dir, dst_dir)
slide_collection.quantify_all_slides()
slide_collection.quantification_results

In [ ]:
src_dir= r"E:\valis_out\N_2016_000021\valis\registered_slides"
dst_dir = r"E:\valis_out\N_2016_000021"
slide_collection = SlideCollection("N_2016_000021", src_dir, dst_dir)
slide_collection.quantify_all_slides()
slide_collection.get_dual_antigen_combinations()
slide_collection.get_triplet_antigen_combinations()
slide_collection.quantification_results

In [ ]:
src_dir= r"E:\valis_out\N_2016_000167\valis\registered_slides"
dst_dir = r"E:\valis_out\N_2016_000167"
slide_collection = SlideCollection("N_2016_000167", src_dir, dst_dir)
slide_collection.quantify_all_slides()
slide_collection.get_dual_antigen_combinations()
slide_collection.get_triplet_antigen_combinations()
slide_collection.quantification_results

In [ ]:
# slide_collection.dual_antigen_results


In [ ]:
slide = slide_collection.slides[5]
slide.properties

In [ ]:
slide_name = "N_2014_000862_ROR2"
slide = [slide for slide in slide_collection.slides if slide.name == slide_name][0]
print(slide.name)

In [ ]:
slide_collection.quantification_results.loc[len(slide_collection.quantification_results)] = slide.quantification_summary

In [ ]:
slide.quantification_summary

In [ ]:
slide_collection.quantification_results

In [ ]:
slide_collection.quantification_results.to_csv(slide_collection.data_dir + '/quantification_results.csv', sep=",",index=False,encoding='utf-8')


In [ ]:
import pickle
pickle.dump(slide.quantification_summary, open(slide_collection.pickle_dir + "/quantification_summary.pickle", "wb"))

In [ ]:
import os
os.path.join(slide_collection.tiles_dir,slide.name,"dab")

In [ ]:
slide_collection.slides[1].summarize_quantification_results()

In [ ]:
slide.quantification_summary['Background (%)']

In [ ]:
slide_collection.quantification_results.loc[len(slide_collection.quantification_results)] = slide.quantification_summary

In [ ]:
slide_collection.quantification_results